In [ ]:
import mgd
import os
import pandas as pd
import torchaudio
import torchaudio.transforms as transforms
import utils
import cnn
import torch.nn as nn
import torch.optim as optim
import s

In [ ]:
AUDIO_DIR = os.path.expanduser("/Users/noramendoza/Desktop/DL_project/data/fma_small")
TRACKS_FILE = utils.load("/Users/noramendoza/Desktop/DL_project/TRACKS.csv")
TRACKS_FILE.head()

Create dataset containing *spectrograms* of song tracks and their *labels*

In [ ]:
music_data = mgd.MusicGenreDataset(AUDIO_DIR, TRACKS_FILE, apply_spectrogram=True)
print(f"There are {len(music_data)} samples in the dataset.")

In [ ]:
s, l, p = music_data[1]

In [ ]:
print(p)
print(l)
print(s)

In [ ]:
utils.plot_spectrogram(s) 

In [ ]:
X = []
y = []

for s,l,p in music_data: 
    X.append(s)
    y.append(l)
    

In [ ]:
len(X)

We are not able to get all 8000 files, but we can take at least 4470

In [ ]:
import torch
from torch.nn.functional import pad

# Determine the maximum size in each dimension
max_size = torch.Size(max(s.size(i) for s in X) for i in range(X[0].ndim))

# Pad all tensors to the max size
X = [pad(t, (0, max_size[-1] - t.size(-1), 0, max_size[-2] - t.size(-2))) for t in X]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on your labels and transform the labels
y_numeric = encoder.fit_transform(y)

# Now you can create a tensor from y_numeric
y = torch.tensor(y_numeric)

In [ ]:
from sklearn.model_selection import train_test_split

# First split the data into a training set and a temporary set using an 80-20 split.
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Then split the temporary set into validation and testing sets using a 50-50 split.
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
X_train[1].shape

In [ ]:
from torch.utils.data import DataLoader
import s

data = X_train # Your data (list of tensors)

# Define DataLoader parameters
batch_size = 32
shuffle = True
num_workers = 4

# Create an instance of your custom dataset
train_custom_dataset = s.final_dataset(data, y_train)
test_dataset = s.final_dataset(X_test, y_test)

# Create a DataLoader
train_dataloader = DataLoader(train_custom_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [ ]:
import model
model.train_model(train_dataloader, test_dataloader)

We have had trouble dealing with the model. At first we worked with a small number of files, files that we first convert to spectrograms, then slice them into (1,128,128) shape and then convert them to tensors. Working with these, and with the existing model, we achieved an accuracy of more than 60%. 

But then we tried the same for the whole set, and since we had trouble splitting the spectrograms, we opt not to do it and therefore now we have this problem. We know where the problem is but we are having much trouble solving it.